# Aula 1 — `if / elif / else` aplicados a dados (pandas-friendly)

**Duração:** 1h  
**Objetivo:** entender condições em Python e como elas viram regras de negócio em um DataFrame (classificações, flags, categorias).

**Dataset do dia (dinâmico):** pedidos de e-commerce com valor, desconto, prazo de envio e canal.


## 1) Setup + criação do dataset (dinâmico)

Nesta aula, vamos **gerar um dataset na hora** para praticar.  
Isso é ótimo para treinar sem depender de arquivos externos.


In [1]:
import numpy as np
import pandas as pd

# Gerador aleatório reprodutível (se você mudar a seed, os dados mudam)
rng = np.random.default_rng(42)

n = 60
df = pd.DataFrame({
    "order_id": np.arange(1, n+1),
    "channel": rng.choice(["site", "app", "marketplace"], size=n, p=[0.45, 0.45, 0.10]),
    "order_value": np.round(rng.uniform(15, 450, size=n), 2),
    "discount_pct": rng.choice([0, 5, 10, 15, 20, 30], size=n, p=[0.35, 0.2, 0.2, 0.1, 0.1, 0.05]),
    "days_to_ship": rng.integers(0, 9, size=n),
})

# Valor pago após desconto
df["paid_value"] = np.round(df["order_value"] * (1 - df["discount_pct"]/100), 2)

df.head(10)

,order_id,channel,order_value,discount_pct,days_to_ship,paid_value
0,1,app,305.76,10,7,275.18
1,2,site,219.93,10,8,197.94
2,3,app,260.88,0,2,260.88
3,4,app,347.77,5,8,330.38
4,5,site,291.10,0,1,291.10
5,6,marketplace,255.81,5,4,243.02
6,7,app,258.26,0,4,258.26
7,8,app,147.22,0,2,147.22
8,9,site,28.41,0,0,28.41
9,10,app,204.97,10,6,184.47


## 2) Entendendo `if / elif / else` na prática

A estrutura básica é:

```python
if condicao:
    ...
elif outra_condicao:
    ...
else:
    ...
```

Em dados, isso vira **regras de negócio**, por exemplo:
- classificar pedidos por faixa de valor
- marcar atrasos
- definir prioridade de atendimento


## 3) Exemplo 1 — Faixa de valor (classificação)

Vamos criar `value_band` para classificar o pedido conforme o **valor pago** (`paid_value`):
- `< 80` → `baixo`
- `< 200` → `medio`
- `>= 200` → `alto`

Aqui vamos usar uma **função** + `.apply()` (coluna → linha a linha).


In [2]:
def value_band(v: float) -> str:
    if v < 80:
        return "baixo"
    elif v < 200:
        return "medio"
    else:
        return "alto"

df["value_band"] = df["paid_value"].apply(value_band)

df[["paid_value", "value_band"]].head(10)

,paid_value,value_band
0,275.18,alto
1,197.94,medio
2,260.88,alto
3,330.38,alto
4,291.10,alto
5,243.02,alto
6,258.26,alto
7,147.22,medio
8,28.41,baixo
9,184.47,medio


## 4) Exemplo 2 — Flag de atraso (condição simples)

Aqui não precisamos de função: uma comparação já cria um booleano (`True/False`).
- `days_to_ship >= 5` → atrasou


In [3]:
df["late_ship"] = df["days_to_ship"] >= 5
df[["days_to_ship", "late_ship"]].head(10)

,days_to_ship,late_ship
0,7,True
1,8,True
2,2,False
3,8,True
4,1,False
5,4,False
6,4,False
7,2,False
8,0,False
9,6,True


## 5) Exemplo 3 — Prioridade combinada (regras em cadeia)

Agora juntamos várias condições. Exemplo de regra:
- se atrasou **e** `paid_value >= 200` → `critica`
- senão, se atrasou → `alta`
- senão, se `paid_value >= 200` → `media`
- senão → `baixa`

Aqui usamos `.apply(..., axis=1)` porque precisamos olhar **mais de uma coluna ao mesmo tempo**.


In [4]:
def priority(row: pd.Series) -> str:
    if row["late_ship"] and row["paid_value"] >= 200:
        return "critica"
    elif row["late_ship"]:
        return "alta"
    elif row["paid_value"] >= 200:
        return "media"
    else:
        return "baixa"

df["priority"] = df.apply(priority, axis=1)

df[["paid_value","days_to_ship","late_ship","value_band","priority"]].head(12)

,paid_value,days_to_ship,late_ship,value_band,priority
0,275.18,7,True,alto,critica
1,197.94,8,True,medio,alta
2,260.88,2,False,alto,media
3,330.38,8,True,alto,critica
4,291.10,1,False,alto,media
5,243.02,4,False,alto,media
6,258.26,4,False,alto,media
7,147.22,2,False,medio,baixa
8,28.41,0,False,baixo,baixa
9,184.47,6,True,medio,alta


## 6) Checagens rápidas (sanity checks)

Vamos ver quantos pedidos caíram em cada categoria.


In [5]:
df["value_band"].value_counts()

,count
value_band,
alto,31
medio,20
baixo,9


In [6]:
df["priority"].value_counts()

,count
priority,
media,20
baixa,18
alta,11
critica,11


## 7) Exercícios (para praticar)

### Exercício 1 — Promoção
Crie uma coluna `is_promo` que seja `True` quando `discount_pct >= 15`.

### Exercício 2 — Taxa por canal
Crie uma coluna `channel_fee_pct`:
- `marketplace`: 12
- `app`: 6
- `site`: 4

### Exercício 3 — Taxa em valor
Calcule `fee_value = paid_value * channel_fee_pct/100`.

### Exercício 4 — Tag de risco
Crie `risk_tag`:
- se `fee_value >= 20` **e** `late_ship == True` → `"risco_alto"`
- se `fee_value >= 20` → `"risco_medio"`
- senão → `"risco_baixo"`

> Dica: você pode usar `if/elif/else` com `df.apply(..., axis=1)` ou fazer em etapas.


In [ ]:
# ✅ Sua resolução aqui (comece por partes)

#1) is_promo
df["is_promo"] = df["discount_pct"] >= 15

# 2) channel_fee_pct
df["channel_fee_pct"].apply(channel_fee)

def channel_fee():
  if channel == "marketplace":
    return 12
  elif channel=="app":
    return 6
  else:
    return 4

# 3) fee_value
# ...

# 4) risk_tag
# ...

df.head(10)

## 8) Desafio extra (opcional)

Crie uma coluna `attention_needed` (boolean) que seja `True` quando:
- `priority` for `"critica"` ou `"alta"`, **ou**
- `discount_pct >= 20`

Depois, mostre somente os pedidos que precisam de atenção.


In [ ]:
# Desafio extra (opcional)
# df["attention_needed"] = ...
# df.loc[df["attention_needed"]].head()